In [1]:
import json
import re

from src.preprocessing.sql_utils import tokenize_sql

In [2]:
perform_tokenise_sql = False

In [22]:
def convert_instance(data):
    output = []
    var_sql = None
    var_sql = data["sql"][0]
    for sentence in data["sentences"]:
        text = sentence['text']
        sql = var_sql # Needed to do variable replacement correctly

        # Variable replacement
        for name in sentence['variables']:
            value = sentence['variables'][name]
            if len(value) == 0:
                for variable in data['variables']:
                    if variable['name'] == name:
                        value = variable['example']
            text = value.join(text.split(name))
            sql = value.join(sql.split(name))

        # Tokenise
        if perform_tokenise_sql:
            sql = tokenize_sql(sql)
        
        output.append(dict(Title=text, QueryBody=sql))
    return output

In [102]:
# data_json_file = "../data/text-to-sql-data/atis.json"
# data_json_file = "../data/text-to-sql-data/academic.json"
data_json_file = "../data/text-to-sql-data/advising.json"
# data_json_file = "../data/text-to-sql-data/geography.json"
# data_json_file = "../data/text-to-sql-data/imdb.json"
# data_json_file = "../data/text-to-sql-data/restaurants.json"
# data_json_file = "../data/text-to-sql-data/scholar.json"
# data_json_file = "../data/text-to-sql-data/yelp.json"

In [103]:
def get_data(json_file):
    with open(json_file) as in_fp:
        data_json = json.load(in_fp)
    train_dev_test = []
    for instance in data_json:
        train_dev_test.extend(convert_instance(instance))
    return train_dev_test

In [104]:
train_dev_test = get_data(data_json_file)

In [105]:
len(train_dev_test)

4387

In [106]:
train_dev_test[0]

{'Title': 'Can undergrads take 550 ?',
 'QueryBody': 'SELECT DISTINCT COURSEalias0.ADVISORY_REQUIREMENT , COURSEalias0.ENFORCED_REQUIREMENT , COURSEalias0.NAME FROM COURSE AS COURSEalias0 WHERE COURSEalias0.DEPARTMENT = "EECS" AND COURSEalias0.NUMBER = 550 ;'}

## Number of SQL n-grams

In [107]:
ngrams = set()
n = 3
for ex in train_dev_test:
    tokens = [t.lower() for t in ex['QueryBody'].split()]
    for i in range(len(tokens)-n+1):
        ngrams.add(tuple(tokens[i:i+n]))
print(len(ngrams))

11259


## Number of question n-grams

In [108]:
ngrams = set()
n = 3
for ex in train_dev_test:
    tokens = [t.lower() for t in ex['Title'].split() if t not in ['.', '?', ',']]  # spider doesn't contain other punctuations AFAIR, but we should replace this with a better filter for other datasets
    for i in range(len(tokens)-n+1):
        ngrams.add(tuple(tokens[i:i+n]))
print(len(ngrams))

20007
